### Imports

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

### Code

In [ ]:
def check_accuracy():
    divide_by_me = [36.110212335692616, 28.847334410339258, 12.040795684423466, 11.747697368421052, 17.566650270536154, 22.937058445728965, 44.474470734744706, 14.57078743370053, 12.691186922530207, 16.211075805719474, 22.76163161249203, 19.283477321814257, 12.777459749552772, 30.787068965517243, 19.05709711846318, 11.576337115072933, 24.4107997265892, 19.611751784733663]
    log_preds, y = learn.TTA()
    preds = np.exp(log_preds)
    probs = np.mean(preds, 0)
        
    thresholded_probs = [np.divide(prob, divide_by_me) for prob in probs]
    print('acc:', accuracy_np(probs, y))
    print('acc with thresholding:', accuracy_np(thresholded_probs, y))

In [ ]:
PATH='data/dups_removed/sample/'
sz = 224

In [5]:
def nasnet(pre): return nasnetalarge(pretrained = 'imagenet' if pre else None)
model_features[nasnet]=4032*2
stats = ([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
tfms = tfms_from_stats(stats, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [6]:
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=2, num_workers=4, test_name=None) 

### Training the last layer

In [7]:
learn = ConvLearner.pretrained(nasnet, data, precompute=True, xtra_fc=[])

In [ ]:
learn.lr_find()
learn.sched.plot()

In [8]:
prefix = 'no-dups'
last_layer_save_name = 'f{prefix}-resnext101-last'
last_layer_best_save_name = f'{prefix}-resnext101-last-best'
last_layer_done_name = f'{prefix}-resnext101-last-layer-done'
ft_name = f'{prefix}-nasnet-ft'
ft_best_name = f'{ft_name}-best'

In [9]:
lr = 5e-4

In [ ]:
learn.fit(lr, 8, 
          cycle_save_name=last_layer_save_name, 
          best_save_name=last_layer_best_save_name)

In [10]:
learn.load(last_layer_best_save_name)

In [11]:
learn.precompute=False

In [ ]:
learn.fit(last_layer_lr, 9, cycle_len=1, cycle_save_name=last_layer_save_name, best_save_name=last_layer_best_save_name)

In [ ]:
#learn.fit(last_layer_lr, 2, cycle_len=1, cycle_save_name=last_layer_save_name, best_save_name=last_layer_best_save_name)

In [ ]:
learn.load(last_layer_best_save_name)
#check_accuracy()

### Fine tuning on all weights

In [12]:
learn.unfreeze()


In [13]:
learn.save(last_layer_done_name)
#learn.load(last_layer_done_name)
#learn.lr_find()
#learn.sched.plot()

In [14]:
lrs = [lr/100, lr/10, lr]

In [ ]:
learn.fit(lrs, 4, cycle_len=1, cycle_mult=2,
          cycle_save_name=ft_name,
          best_save_name=ft_best_name)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

HBox(children=(IntProgress(value=0, max=2214), HTML(value='')))

/Users/alex/Git_Files/shopee_com/fastai/model.py:49: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  return raw_loss.data[0]


epoch      trn_loss   val_loss   accuracy   
    0      4.222384   14.609429  0.072169  


HBox(children=(IntProgress(value=0, max=2214), HTML(value='')))

...

HBox(children=(IntProgress(value=0, max=2214), HTML(value='')))

    3      3.634778   13.984936  0.06292   


HBox(children=(IntProgress(value=0, max=2214), HTML(value='')))

    4      3.457533   15.643769  0.057876  


HBox(children=(IntProgress(value=0, max=2214), HTML(value='')))

In [ ]:
learn.load(ft_best_name)
check_accuracy() #pb = 0.7866125229237622

In [ ]:
def generate_csv(name):
    divide_by_me = [34.74545454545454, 29.542028985507248, 12.324062877871826, 
             11.278495020287716, 18.092307692307692, 23.61081081081081, 
             47.18518518518518, 14.422641509433962, 12.787954830614806,
             16.689956331877728, 22.75, 19.984313725490196, 
             13.105872267466781, 26.40414507772021, 19.841661258922777, 
             11.985887887103097, 21.0, 19.351898734177215]

    t_log_preds, y = learn.TTA(is_test=True)
    probs = np.exp(t_log_preds)
    probs = np.mean(probs, 0)
    thresholded_probs = [np.divide(prob, divide_by_me) for prob in probs]
    cats = np.argmax(thresholded_probs, 1)
    df = pd.DataFrame(cats)
    df.columns = ['category']
    df.insert(0, 'id', [o[10:-4] for o in data.test_ds.fnames])
    df.head()
    SUBM = f'{PATH}subm/'
    os.makedirs(SUBM, exist_ok=True)
    df.to_csv(f'{SUBM}{name}.gz', compression='gzip', index=False)

In [ ]:
generate_csv('test-no-dup')

###  Analyzing results

In [ ]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)

In [ ]:
print(probs.shape)
probs[0]

In [ ]:
preds = np.argmax(probs, axis=1)
probs = probs[:,1]

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, preds)

In [ ]:
plot_confusion_matrix(cm, (range(1,19)))

In [ ]:
print('Bad categories')
print(data.classes[11], data.classes[7])
print(data.classes[12], data.classes[10])
print(data.classes[9], data.classes[15])
print(data.classes[6], data.classes[8])

In [ ]:
lowest_probs_ranked = np.argsort(probs)
print(lowest_probs_ranked)

In [ ]:
def rand_by_mask(mask): return np.random.choice(np.where(mask)[0], 4, replace=False)
def rand_by_correct(is_correct): return rand_by_mask((preds==data.val_y)==is_correct)
def load_img_id(ds, idx): return PIL.Image.open(PATH+ds.fnames[idx])
def plot_val_with_title(idxs, title):
    imgs = [load_img_id(data.val_ds,x) for x in idxs]
    title_probs = [probs[x] for x in idxs]
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8))

In [ ]:
plot_val_with_title(lowest_probs_ranked[:10], 'worst failures')
print(y[lowest_probs_ranked[:10]])
print(preds[lowest_probs_ranked[:10]])
print(data.classes)
print(lowest_probs_ranked[:10])
'''
def most_by_mask(mask, mult):
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]
def most_by_correct(y, is_correct):
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask(((preds == data.val_y)==is_correct) & (data.val_y==y), mult)
'''

In [ ]:
!open .

### Extensions

In [ ]:
'''
3) add some dropout (do we train from scratch?, i think so..)
3) try resnext 
4) try all data
'''
'''
!) Add more transforms
2) try super resolution 
'''